In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time # check the processing overhead
import torch.optim.lr_scheduler as lr_scheduler

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])




==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Preparing data..


In [7]:
class HW_6_Net(nn.Module):
    def __init__(self):
        super(HW_6_Net, self).__init__()

        self.c1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5, stride=2, padding=1)
        self.c2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1)
        self.c3 = nn.Conv2d(in_channels=64, out_channels=192, kernel_size=3, padding=1)
        self.c4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=2, padding=1)
        self.inception_1 = Inception_1()
        self.inception_2 = Inception_2()
        #self.f1 = nn.Linear(256, 10)
        self.f1 = nn.Linear(256, 10)

        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x1 = F.relu(self.c1(x))
        x1 = self.dropout(x1)
        
        x2 = F.relu(self.c2(x1))
        x2 = self.dropout(x2)
           
        x3 = F.relu(self.c3(x2)) 
        x3_maxpool = F.max_pool2d(x3, kernel_size=3, stride=2, padding=1) #8x8x64
        x3_maxpool = self.dropout(x3_maxpool)
        
        x4 = F.relu(self.inception_1(x3_maxpool))
        x4 = self.dropout(x4)
  
        x5 = F.relu(self.inception_2(x4))
        x5_maxpool = F.max_pool2d(x5, kernel_size=3, stride=2, padding=1) #4x4x256
        x5_maxpool = self.dropout(x5_maxpool)

        x6 = F.relu(self.c4(x5_maxpool))
        x6 = self.dropout(x6)
 
        x6_avgpool = F.avg_pool2d(x6, kernel_size=2, stride=2, padding=0)
        x6_avgpool_dropped = self.dropout(x6_avgpool)
        
        x7 = torch.flatten(x6_avgpool, 1)
    
        x8 = F.relu(self.f1(x7))
        
        return x8
        
        '''
        x4 = F.relu(self.inception_1(x3_maxpool))
        x4_maxpool = F.max_pool2d(x3, kernel_size=3, stride=2, padding=1) #4x4x256
                
        x5 = F.relu(self.c4(x4_maxpool))
        x5_avgpool = F.avg_pool2d(x5, kernel_size=4, stride=4, padding=0)
        x5_avgpool_dropped = self.dropout(x5_avgpool)
        
        x6 = torch.flatten(x5_avgpool, 1)
        
        x7 = F.relu(self.f1(x6))
        
        return x7
        '''
        
        '''
        x4 = F.relu(self.c4(x3_maxpool))
        x4_avgpool = F.max_pool2d(x4, kernel_size=4, stride=4, padding=1)
        x4_avgpool_dropped = self.dropout(x4_avgpool)
        
        x5 = torch.flatten(x4_avgpool_dropped, 1)
        
        x6 = F.relu(self.f1(x5))
        
        return x6
        '''
        
        '''
        x4 = F.relu(self.inception_1(x3))

        x4_maxpool = F.max_pool2d(x4, kernel_size=3, stride=2, padding=1)
        
        x5 = F.relu(self.inception_2(x4_maxpool))
        x5_maxpool = F.max_pool2d(x5, kernel_size=3, stride=2, padding=1)
        
        x6 = F.relu(self.inception_2(x5_maxpool))
        x6_avgpool = F.avg_pool2d(x6,  kernel_size=3, stride=3, padding=0)
        x6_avgpool_dropped = self.dropout(x6_avgpool)

        x7 = torch.flatten(x6_avgpool_dropped, 1)

        x8 = F.relu(self.f1(x7))

        return x8
        '''
        
'''
        x5 = F.relu(self.inception_2(x4))
        x5_maxpool = F.max_pool2d(x5, kernel_size=3, stride=2, padding=1)

        x6 = F.relu(self.inception_2(x5_maxpool))

        x7 = F.relu(self.inception_2(x6))
        x7_maxpool = F.max_pool2d(x7, kernel_size=3, stride=2, padding=1)

        x8 = F.relu(self.inception_2(x7_maxpool))
        x8_avgpool = F.avg_pool2d(x8, kernel_size=5, stride=3, padding=1)
        x8_avgpool_dropped = self.dropout(x8_avgpool)

        x9 = torch.flatten(x8_avgpool_dropped, 1)

        x10 = F.relu(self.f1(x9))

        return x10
'''

class Inception_1(nn.Module):
    def __init__(self):
        super(Inception_1, self).__init__()
        # Branch 1
        self.b1_1 = nn.Conv2d(in_channels=192, out_channels=64, kernel_size=1)
        # Branch 2
        self.b2_1 = nn.Conv2d(in_channels=192, out_channels=96, kernel_size=1)
        self.b2_2 = nn.Conv2d(in_channels=96, out_channels=128, kernel_size=3, padding=1)
        # Branch 3
        self.b3_1 = nn.Conv2d(in_channels=192, out_channels=16, kernel_size=1)
        self.b3_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, padding=2)
        # Branch 4
        self.b4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.b4_2 = nn.Conv2d(in_channels=192, out_channels=32, kernel_size=1)

    def forward(self, x):
        b1 = F.relu(self.b1_1(x))
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))
        b4 = F.relu(self.b4_2(self.b4_1(x)))
        return torch.cat((b1, b2, b3, b4), dim=1)

class Inception_2(nn.Module):
    def __init__(self):
        super(Inception_2, self).__init__()
        # Branch 1
        self.b1_1 = nn.Conv2d(in_channels=256, out_channels=64, kernel_size=1)
        # Branch 2
        self.b2_1 = nn.Conv2d(in_channels=256, out_channels=96, kernel_size=1)
        self.b2_2 = nn.Conv2d(in_channels=96, out_channels=128, kernel_size=3, padding=1)
        # Branch 3
        self.b3_1 = nn.Conv2d(in_channels=256, out_channels=16, kernel_size=1)
        self.b3_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, padding=2)
        # Branch 4
        self.b4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.b4_2 = nn.Conv2d(in_channels=256, out_channels=32, kernel_size=1)

    def forward(self, x):
        b1 = F.relu(self.b1_1(x))
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))
        b4 = F.relu(self.b4_2(self.b4_1(x)))
        return torch.cat((b1, b2, b3, b4), dim=1)

def train(model, device, train_loader, optimizer, epoch):
    model.train() # set the model into training model (evaluation model in the testing)
    count = 0

    loss_Fn=nn.CrossEntropyLoss()
    #nn.functional.cross_entropy()
    for batch_idx, (data, target) in enumerate(train_loader):
      #data is the image
      #target is the ground truth
        data, target = data.to(device), target.to(device)

        Predict=model(data)
        loss=loss_Fn(Predict,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 60
    lr = 0.02
    no_cuda = False
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = HW_6_Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    # Define the StepLR scheduler
    step_size = 10
    gamma = 0.7
    scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time()
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))
if __name__ == '__main__':
    main()

Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.300150
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.299669
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.303070
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.302952
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.301511
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.300803
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.301545
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.303104
Train Epoch: 1 [10240/50000 (20%)]	Loss: 2.301687
Train Epoch: 1 [11520/50000 (23%)]	Loss: 2.302923
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.300956
Train Epoch: 1 [14080/50000 (28%)]	Loss: 2.301575
Train Epoch: 1 [15360/50000 (31%)]	Loss: 2.303988
Train Epoch: 1 [16640/50000 (33%)]	Loss: 2.300075
Train Epoch: 1 [17920/50000 (36%)]	Loss: 2.302090
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.302292
Train Epoch: 1 [20480/50000 (41%)]	Loss: 2.301098
Train Epoch: 1 [21760/50000 (43%)]	Loss: 2.305146
Train Epoch: 1 [23040/50000 (46%)]	Loss